## OpenCV 영상에서 남녀구분

In [13]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

#----------------------------------
# 티처블머신 모델 및 라벨 불러오기
#----------------------------------
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("./converted_keras/keras_Model.h5", compile=False)

# Load the labels
class_names = open("./converted_keras/labels.txt", "r").readlines()



#-------------------------------------
# 성별구분(gender) 함수 정의
#-------------------------------------
def gender(imgArea):
    # Create the array of the right shape to feed into the keras model
    # The 'length' or number of images you can put into the array is
    # determined by the first position in the shape tuple, in this case 1
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Replace this with the path to your image
    image = Image.open(imgArea).convert("RGB")

    # resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

    # turn the image into a numpy array
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Load the image into the array
    data[0] = normalized_image_array

    # Predicts the model
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    
    return class_name[2:].strip(),confidence_score



#-----------------------------------------------------------------------
#오픈CV로 쉽게 얼굴 인식하고 성별 구분 함수 적용
#----------------------------------------------------------------------

# haarcascade 불러오기
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#----------카메라 켜기---
# CAMERA can be 0 or 1 based on default camera of your computer
camera = cv2.VideoCapture(0)

while True:
    # Grab the webcamera's image.
    ret, image = camera.read()
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 얼굴 찾기
    faces = face_cascade.detectMultiScale(gray,1.2,5)
    predict=[]
    #print(faces)
    cnt=0
    for (x, y, w, h) in faces:

        #cv2.rectangle(image, (x-20, y-20, w+40, y + 40), (255, 0, 0), 2)
        px=50
        roi_color = image[y-px:y + h+px, x-px:x + w+px]
        cv2.imwrite('tmp.jpg',roi_color)
        pre=gender('tmp.jpg') #성별구분

        predict.append(pre)
        
    #---------------------------
    # 성별 현황집계
    #---------------------------
    df=pd.DataFrame(predict,columns=['성별','비율'])
    tot=df['성별'].value_counts()
#     #현황 텍스트
#     txt=f'total count => male:{tot["male"]}, female: {tot["female"]}'

    
    # 폰트 색상 지정
    blue = (255, 0, 0)
    green= (0, 255, 0)
    red= (0, 0, 255)
    white= (255, 255, 255) 
    font =  cv2.FONT_HERSHEY_PLAIN

#     img = cv2.putText(image, txt, (10, 30), font, 2, blue, 1, cv2.LINE_AA)
    px=50
    for (x, y, w, h),result in zip(faces,predict):
        cv2.rectangle(image, (x-50, y-50, w+px*2, h+px*2), (255, 0, 0), 2)
        tmp_txt=f'{result[0]}: {result[1]:.2f}'
        img = cv2.putText(image, tmp_txt, (x, y-(px+30)), font, 1, blue, 1, cv2.LINE_AA)



    # Show the image in a window
    cv2.imshow("Webcam Image", image)


    # Listen to the keyboard for presses.
    keyboard_input = cv2.waitKey(1)

    # 27 is the ASCII for the esc key on your keyboard.
    if keyboard_input == 27:
        break

camera.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 24ms/step
